Intro
    This notebook serves as a demonstration to the data validation functions created for this project.
    This notebook is for the project update two

    1) Validation of the data set. Checking for "hidden" issues in the data
        - Outliers
        - Dupplicate Data
        - Balance
        - Correlation

    2) Validation of the Train / Test Split
        - Tests that a dataset is representative of the master dataset

In [1]:
import pandas as pd
import numpy as  np

from scipy.stats import ks_2samp

import warnings
warnings.filterwarnings('ignore')

# Data Preprocessing

## Helpers

In [2]:
def find_outliers(data_df, categorical_threshold=0.1):
    """
    :param data_df: Pandas DataFrame Containing the Data in question
    :param categorical_threshold: The frequency threshold to mark a categorical value as an outlier
    :return: A Pandas DataFrame listing the index of all outlier values

    This function cycles through the column in the provided data and looks for outliers
    There are two checks that are preformed.
        1) Checking if a categorical column contains any values that fall below the acceptable frequency threshold and thus an outlier
        2) Checking if a numerical column contains any statistical outliers, identified using IQR values
    """

    outlier_df = pd.DataFrame(columns=["Column","Value","Index","Issue"])
    for col in data_df.columns:
        if data_df[col].dtype == 'object' and all(isinstance(val, str) for val in data_df[col]):
            freq = data_df[col].value_counts(normalize=True)
            outlier_values = freq[freq < categorical_threshold].index
            outliers = data_df[data_df[col].isin(outlier_values)]
        elif data_df[col].dtype in ["float64","int64"]:
            q1 = data_df[col].quantile(0.25)
            q3 =  data_df[col].quantile(0.75)
            iqr = q3 - q1
            outliers = data_df[(data_df[col] < (q1 - 1.5 * iqr)) | (data_df[col] > (q3 + 1.5 * iqr))][col]
        else:
            pass

        # Prepare the temp "outliers" DF to be concatenated with the main outlier_df
        outliers = outliers.reset_index()
        outliers["Column"] = col
        outliers = outliers.rename(columns={"index":"Index",col:"Value"})
        outliers = outliers[["Column","Value","Index"]]
        outliers["Issue"] = "Outlier"
        outlier_df = pd.concat([outlier_df, outliers])

        return outlier_df

In [3]:
def find_duplicates(data_df):
    """
    :param data_df: Pandas DataFrame Containing the Data in question
    :return: A Pandas DataFrame listing the index of all duplicate values
    """

    duplicates = data_df[data_df.duplicated()]
    duplicates["Column"] = str(duplicates.columns.values)
    duplicates["Value"] = "N/A"
    duplicates = duplicates.reset_index()
    duplicates = duplicates.rename(columns={"index":"Index"})
    duplicates["Issue"] = "Duplicates"
    duplicates = duplicates[["Column","Value","Index","Issue"]]

    return duplicates

In [4]:
def find_balance(data_df, label_field):
    """
    :param data_df: Pandas DataFrame Containing the Data in question
    :param label_field: String value of the label field
    :return: A Pandas DataFrame listing the class imbalance for each unique label
    """

    label_count = data_df.groupby(label_field).size().reset_index(name="label_count")
    total_count = label_count["label_count"].sum()
    label_count["label_percent"] = (label_count["label_count"]/total_count)*100

    label_count["imbalance_level"] = np.where(
        (label_count["label_percent"] > 20) & (label_count["label_percent"] <= 40)
        # A label being 20%-40$ of the data is mild imbalance
        ,"Mild"
        ,np.where(
            (label_count["label_percent"] > 1) & (label_count["label_percent"] <= 20)
            # A label being 1%-20% of the data is moderate imbalance
            ,"Moderate"
            ,np.where(
                (label_count["label_percent"] <= 1)
                # A label being <1% of the data is an extreme imbalance
                ,"Extreme"
                # This is the 'else' for units 40%-100% of the data, which is not an imbalance
                ,"None"
            )
        )
    )

    return label_count

In [5]:
def find_correlation(data_df, corr_method="pearson", threshold = 0.75):
    
    correlation = data_df.corr(method = corr_method)
    
    correlation = correlation.unstack().reset_index()
    
    correlation = correlation[correlation["level_0"]!=correlation["level_1"]]
    
    correlation = correlation[correlation[0] >= threshold]
    
    correlation = correlation.rename(columns={"level_0":"Attribute_1","level_1":"Attribute_2",0:"Absolute_Correlation"})
    
    return correlation

## Wrapper

In [6]:
def validate_dataset(data, categorical_threshold, label_field=None, corr_method="pearson", threshold = 0.75, data_type="excel"):

    # First step is to determine what was passed in and convert to a DF
    if data_type == "excel":
        data_df = pd.read_excel(data)
    elif data_type == "csv":
        data_df = pd.read_csv(data)
    elif data_type == "df":
        data_df = data
    else:
        return "Unexpected Type"

    # Check 1: Find Outliers
    outliers = find_outliers(data_df, categorical_threshold=categorical_threshold)

    # Check 2:
    duplicates = find_duplicates(data_df)
    
    # Check 3:
    if label_field is not None:
        balance = find_balance(data_df, label_field)
    
    # Check 4:
    correlation = find_correlation(data_df,corr_method=corr_method,threshold=threshold)

    if label_field is not None:
        return {"outliers":outliers,"duplicates":duplicates,"balance":balance,"correlation":correlation}
    else:
        return {"outliers":outliers,"duplicates":duplicates,"correlation":correlation}


# Model Training & Testing

In [7]:
def test_representative(dataset, sample):
    return_df = pd.DataFrame(columns=["Feature","P_Value","Issue"])
    for feature in dataset.columns:
        ks_stat, p_value = ks_2samp(dataset[feature], sample[feature])
        # Distributions are not significantly different
        if p_value >= 0.05:
            return_df = pd.concat([return_df,pd.DataFrame([[feature,p_value,"Representative"]],columns=["Feature","P_Value","Issue"])])
        # Distributions are significantly different
        else:
            return_df = pd.concat([return_df,pd.DataFrame([[feature,p_value,"Not Representative of Dataset"]],columns=["Feature","P_Value","Issue"])])

    return return_df

# Demonstrate Functions 

## Get Test Data

In [8]:
test_1 = pd.read_csv("test_repos/linear-regression-example/data/Advertising_data.csv")

In [9]:
test_1.head(2)

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,2210
1,44.5,39.3,45.1,1040


In [10]:
data = {'col1': ['a', 'b', 'c', 'd', 'e', 'a','a','a','f'],
        'col2': [1, 2, 3, 4, 5, 1, 1, 1, 2]}
test_2 = pd.DataFrame(data)

In [11]:
test_2.head(2)

,col1,col2
0,a,1
1,b,2


## Use Functions

    Checking for outliers in a categorical column

In [12]:
find_outliers(test_2, categorical_threshold=0.15)

,Column,Value,Index,Issue
0,col1,b,1,Outlier
1,col1,c,2,Outlier
2,col1,d,3,Outlier
3,col1,e,4,Outlier
4,col1,f,8,Outlier


    Checking for duplicates

In [13]:
find_duplicates(test_2)

,Column,Value,Index,Issue
0,['col1' 'col2'],N/A,5,Duplicates
1,['col1' 'col2'],N/A,6,Duplicates
2,['col1' 'col2'],N/A,7,Duplicates


    Check for imbalance

In [14]:
find_balance(test_2, label_field="col2")

,col2,label_count,label_percent,imbalance_level
0,1,4,44.444444,None
1,2,2,22.222222,Mild
2,3,1,11.111111,Moderate
3,4,1,11.111111,Moderate
4,5,1,11.111111,Moderate


    find_correlation

In [15]:
find_correlation(test_1, corr_method="pearson", threshold = 0.75)

,Attribute_1,Attribute_2,Absolute_Correlation
3,TV,sales,0.782224
12,sales,TV,0.782224


    Check if representative

In [16]:
test_representative(test_1, test_1.sample(50))

,Feature,P_Value,Issue
0,TV,0.641882,Representative
0,radio,0.922029,Representative
0,newspaper,0.844801,Representative
0,sales,0.798101,Representative
